## Load the data

To begin, load up some data from a database into a DataFrame. You'll use data from Kickstarter in this lesson.

In [0]:
# To keep the page organized, do all imports here
import pandas as pd
from scipy import stats

projects_df = pd.read_csv("https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/ksprojects.csv")

Now, quickly examine your DataFrame.

In [2]:
projects_df.shape

(65418, 13)

In [3]:
projects_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65418 entries, 0 to 65417
Data columns (total 13 columns):
id               65418 non-null int64
name             65418 non-null object
category         65418 non-null object
main_category    65418 non-null object
deadline         65418 non-null object
goal             65418 non-null float64
launched         65418 non-null object
state            65418 non-null object
backers          65418 non-null int64
country          65418 non-null object
usd_pledged      64673 non-null float64
currency         65418 non-null object
pledged          65418 non-null float64
dtypes: float64(3), int64(2), object(8)
memory usage: 6.5+ MB


You can see that there are several numeric columns and a couple of date columns. There's also some categorical data. Most importantly, though, there are no nulls in any columns except *usd_pledged*. Notice that each column has 65,418 values, except *usd_pledged*, which has 64,673 values. If you are going to use that column in your analysis, you'll need to deal with those nulls.  



In [4]:
projects_df.head()

,id,name,category,main_category,deadline,goal,launched,state,backers,country,usd_pledged,currency,pledged
0,1147015301,"""All We Had"" Gets Into Cannes -- $10 or More G...",Documentary,Film & Video,2009-05-20 21:50:00,300.0,2009-04-30 22:10:30,failed,4,US,40.00,USD,40.00
1,1100844465,daily digest,Documentary,Film & Video,2009-06-01 02:20:00,700.0,2009-05-04 21:14:28,successful,14,US,700.00,USD,700.00
2,1316334968,Drive A Faster Car 2.0,Nonfiction,Publishing,2009-06-04 06:00:00,1000.0,2009-05-04 22:51:31,successful,32,US,1367.00,USD,1367.00
3,1304906577,Accidental to Edinburgh - PHASE 1: AIRFARE,Theater,Theater,2009-06-05 05:59:00,6000.0,2009-04-30 22:22:43,successful,24,US,6575.00,USD,6575.00
4,1099226462,Logical Guess Pictures' 2nd Horror Movie!,Film & Video,Film & Video,2009-06-06 00:45:00,500.0,2009-04-30 01:32:55,successful,22,US,501.66,USD,501.66


Before you move on, it would be interesting to find out how many different categories exist. There are a few ways to do this.

In [5]:
# Count the number of unique values in this column
projects_df['category'].nunique()

158

In [6]:
# Find the frequency of each value in the column
category_counts = projects_df['category'].value_counts()

# Only print the first 10, because 158 is too many to print
category_counts.head(10)

Product Design    3444
Documentary       2969
Music             2740
Shorts            2407
Tabletop Games    2208
Food              2126
Video Games       2011
Film & Video      1828
Fiction           1670
Fashion           1584
Name: category, dtype: int64

Do the same for the *main_category* column.

In [7]:
projects_df['main_category'].nunique()

15

In [8]:
main_category_counts = projects_df['main_category'].value_counts()

# There are only 15 main categories, so print them all.
main_category_counts

Film & Video    11666
Music            9395
Publishing       6901
Games            5739
Technology       5328
Art              4843
Design           4798
Food             4343
Fashion          3700
Theater          2015
Photography      1933
Comics           1791
Crafts           1474
Journalism        811
Dance             681
Name: main_category, dtype: int64

The *state*, *currency*, and *country* columns also look like they have categorical data. See if there's anything interesting going on there.

In [9]:
# How many countries?
projects_df['country'].nunique()

22

In [10]:
# How are the projects distributed over the countries?
country_counts = projects_df['country'].value_counts()
country_counts

US      52071
GB       5617
CA       2404
AU       1258
N,"0      744
DE        538
NL        457
FR        397
IT        374
ES        294
SE        267
NZ        236
DK        157
NO        120
IE         99
AT         98
BE         90
CH         88
MX         43
SG         29
HK         26
LU         11
Name: country, dtype: int64

So although the majority of projects are from the US, there are many other countries represented. Now, examine the currency.

In [11]:
projects_df['currency'].nunique()

13

In [12]:
currency_counts = projects_df['currency'].value_counts()
currency_counts

USD    52597
GBP     5722
CAD     2445
EUR     2399
AUD     1271
SEK      271
NZD      241
DKK      163
NOK      123
CHF       88
MXN       43
SGD       29
HKD       26
Name: currency, dtype: int64

As expected, most projects use US dollars. However, many others use a different currency. This is an important discovery. Before you do any sort of analysis on money, you need to ensure that you are working with the same currency. Trying to find the mean would be meaningless if the amounts are in different currencies. The *usd_pledged* column is the amount of money pledged converted to USD, but the *pledged* and *goal* columns are in the specified currency. Before you solve that problem, see how many different states are in the *state* column.


In [13]:
projects_df['state'].nunique()

6

In [14]:
state_counts = projects_df['state'].value_counts()
state_counts

failed        33922
successful    23125
canceled       6427
live            938
undefined       695
suspended       311
Name: state, dtype: int64

## Handling nulls

Earlier, you saw that one of the columns has some null values. If that column is significant to your analysis, then you need to do something about those null values. There are several options:

 - Set a default value
 - Try to "guess" an appropriate value
 - Delete the rows altogether
 
Of course, the choice depends on the data and the type of analysis that you wish to perform. In this example, the nulls occur in the *usd_pledged* column. It might help if you looked at those rows and see what that data looks like.
 
Pandas has a built-in `isnull()` method to help find nulls across all columns in the DataFrame.

In [15]:
null_rows = projects_df[projects_df.isnull().any(axis=1)] # Will return rows that have a null value in any column
null_rows.head(10)

,id,name,category,main_category,deadline,goal,launched,state,backers,country,usd_pledged,currency,pledged
9490,1189637889,"""Natural Born Hooker""",Theater,Theater,2012-04-24 08:59:00,25000.0,2012-03-14 19:05:44,undefined,0,"N,""0",NaN,USD,2621.00
15111,1244477824,Statpedia - The Collaborative Search Engine fo...,Software,Technology,2012-12-14 10:48:14,500.0,2012-11-14 10:48:14,undefined,0,"N,""0",NaN,USD,500.00
17484,1084993156,"Willy Tea, Chris Doud and Chums: The Children'...",Music,Music,2013-04-09 06:06:11,3750.0,2013-03-26 05:06:11,successful,0,"N,""0",NaN,USD,4593.00
17532,1379649454,"Juiette Z. Payne presents debut EP ""Eternal Da...",Music,Music,2013-04-11 15:50:25,5000.0,2013-03-02 15:50:25,successful,0,"N,""0",NaN,USD,5250.00
27334,1349042579,O'Films Production Launch!,Film & Video,Film & Video,2014-04-30 02:58:25,6000.0,2014-04-08 02:58:25,undefined,0,"N,""0",NaN,USD,6168.00
27443,1205956740,"Short Film - ""The Man Who Choked""",Film & Video,Film & Video,2014-05-02 12:47:00,1000.0,2014-04-16 13:21:21,undefined,0,"N,""0",NaN,GBP,1042.00
27522,1125500286,All About Amy,Film & Video,Film & Video,2014-05-04 17:41:13,3000.0,2014-04-04 17:41:13,undefined,0,"N,""0",NaN,USD,3225.00
27574,1292835145,WonderQuest ~ A Feature Length Music Film,Film & Video,Film & Video,2014-05-06 05:12:41,1111.0,2014-04-05 05:12:41,undefined,0,"N,""0",NaN,USD,3032.88
27602,1092574678,Pixelate Film Festival 2014,Film & Video,Film & Video,2014-05-07 13:37:16,1500.0,2014-04-09 13:37:16,undefined,0,"N,""0",NaN,GBP,1552.00
27755,1061706792,Zombie Shuffle feature film,Film & Video,Film & Video,2014-05-12 05:59:00,5000.0,2014-04-09 16:12:49,undefined,0,"N,""0",NaN,USD,5305.00


In [16]:
null_rows.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 745 entries, 9490 to 59365
Data columns (total 13 columns):
id               745 non-null int64
name             745 non-null object
category         745 non-null object
main_category    745 non-null object
deadline         745 non-null object
goal             745 non-null float64
launched         745 non-null object
state            745 non-null object
backers          745 non-null int64
country          745 non-null object
usd_pledged      0 non-null float64
currency         745 non-null object
pledged          745 non-null float64
dtypes: float64(3), int64(2), object(8)
memory usage: 81.5+ KB


In a real-life situation, you may be able to go back to the source of this data and find out why the data is like this. For example, the *country* column appears to be broken, even though it contains a value. And the state seems to be undefined for many of these rows. There are only 745 problem rows, so it may be better for your analysis if you dropped those rows altogether.

If you choose to place a default value in these columns, you could use the `fillna()` method to replace all `NaN` values with a specific value. For example, to replace all `NaN` values with `0`, you could run the following code:

In [17]:
zero_filled = null_rows.fillna(0)
zero_filled.head(10)

,id,name,category,main_category,deadline,goal,launched,state,backers,country,usd_pledged,currency,pledged
9490,1189637889,"""Natural Born Hooker""",Theater,Theater,2012-04-24 08:59:00,25000.0,2012-03-14 19:05:44,undefined,0,"N,""0",0.0,USD,2621.00
15111,1244477824,Statpedia - The Collaborative Search Engine fo...,Software,Technology,2012-12-14 10:48:14,500.0,2012-11-14 10:48:14,undefined,0,"N,""0",0.0,USD,500.00
17484,1084993156,"Willy Tea, Chris Doud and Chums: The Children'...",Music,Music,2013-04-09 06:06:11,3750.0,2013-03-26 05:06:11,successful,0,"N,""0",0.0,USD,4593.00
17532,1379649454,"Juiette Z. Payne presents debut EP ""Eternal Da...",Music,Music,2013-04-11 15:50:25,5000.0,2013-03-02 15:50:25,successful,0,"N,""0",0.0,USD,5250.00
27334,1349042579,O'Films Production Launch!,Film & Video,Film & Video,2014-04-30 02:58:25,6000.0,2014-04-08 02:58:25,undefined,0,"N,""0",0.0,USD,6168.00
27443,1205956740,"Short Film - ""The Man Who Choked""",Film & Video,Film & Video,2014-05-02 12:47:00,1000.0,2014-04-16 13:21:21,undefined,0,"N,""0",0.0,GBP,1042.00
27522,1125500286,All About Amy,Film & Video,Film & Video,2014-05-04 17:41:13,3000.0,2014-04-04 17:41:13,undefined,0,"N,""0",0.0,USD,3225.00
27574,1292835145,WonderQuest ~ A Feature Length Music Film,Film & Video,Film & Video,2014-05-06 05:12:41,1111.0,2014-04-05 05:12:41,undefined,0,"N,""0",0.0,USD,3032.88
27602,1092574678,Pixelate Film Festival 2014,Film & Video,Film & Video,2014-05-07 13:37:16,1500.0,2014-04-09 13:37:16,undefined,0,"N,""0",0.0,GBP,1552.00
27755,1061706792,Zombie Shuffle feature film,Film & Video,Film & Video,2014-05-12 05:59:00,5000.0,2014-04-09 16:12:49,undefined,0,"N,""0",0.0,USD,5305.00


But in this case, you would prefer to drop all the rows with `NaN`. So instead, use the `dropna()` method.

In [18]:
# Print the original size of the DataFrame
print(projects_df.size)

# Drop `NaN` rows
clean_projects = projects_df.dropna()

# Print the size of the modified DataFrame
print(clean_projects.size)

850434
840749


## Statistics

Now that you have a clean DataFrame and you understand some of the categorical data better, turn your attention to the numerical columns. Typically, you want to determine the range of values and the type of distribution of the numbers. Pandas has a few methods to perform many of these tasks on a Series.

You could write a simple function that will print these details for a given Series.

In [0]:
def series_statistics(column):
  print('statistics for column: {}'.format(column.name))
  print('The max value in the column: {}'.format(column.max()))
  print('The min value in the column: {}'.format(column.min()))
  print('The mode value in the column: {}'.format(column.mode()))
  print('The median value in the column: {}'.format(column.median()))
  print('The mean of the column: {}'.format(column.mean()))
  print('The std of the column: {}'.format(column.std()))

In [20]:
series_statistics(clean_projects['goal'])

statistics for column: goal
The max value in the column: 100000000.0
The min value in the column: 1.0
The mode value in the column: 0    5000.0
dtype: float64
The median value in the column: 5000.0
The mean of the column: 42094.83745674393
The std of the column: 1033205.2738987174


In [21]:
series_statistics(clean_projects['usd_pledged'])

statistics for column: usd_pledged
The max value in the column: 8596480.0
The min value in the column: 0.0
The mode value in the column: 0    0.0
dtype: float64
The median value in the column: 542.0
The mean of the column: 7900.242114883215
The std of the column: 72783.34386006395


Because these values are so commonly needed, there is a special function named `describe()` that automatically calculates them all for you, including the interquartile ranges.

In [22]:
clean_projects['goal'].describe()

count    6.467300e+04
mean     4.209484e+04
std      1.033205e+06
min      1.000000e+00
25%      2.000000e+03
50%      5.000000e+03
75%      1.500000e+04
max      1.000000e+08
Name: goal, dtype: float64

In [23]:
clean_projects['usd_pledged'].describe()

count    6.467300e+04
mean     7.900242e+03
std      7.278334e+04
min      0.000000e+00
25%      2.500000e+01
50%      5.420000e+02
75%      3.608380e+03
max      8.596480e+06
Name: usd_pledged, dtype: float64

In fact, the `describe()` method may be applied to an entire DataFrame. All numeric columns in the DataFrame will be analyzed.

In [24]:
clean_projects[['goal', 'usd_pledged', 'backers']].describe()

,goal,usd_pledged,backers
count,6.467300e+04,6.467300e+04,64673.000000
mean,4.209484e+04,7.900242e+03,104.336091
std,1.033205e+06,7.278334e+04,994.725411
min,1.000000e+00,0.000000e+00,0.000000
25%,2.000000e+03,2.500000e+01,2.000000
50%,5.000000e+03,5.420000e+02,13.000000
75%,1.500000e+04,3.608380e+03,56.000000
max,1.000000e+08,8.596480e+06,154926.000000


## Correlations

Sometimes, it's useful to examine if there are correlations between columns. That is, is there a linear relationship between two columns? To perform such a test, use the `corr()` method. This method calculates the *Pearson correlation coefficient* between the two columns. This coefficient is a number between `-1` and `1`. A `1` means that a strong relationship exists, a `-1` means that a strong negative relationship exists, and a `0` means that there is no relationship at all.

In [25]:
clean_projects['goal'].corr(clean_projects['usd_pledged'])

0.006423740131433384

This value is close to `0`, which implies that there is no relationship between the goal and the amount pledged. Are there relationships between the other fields? You can apply this method to the entire DataFrame.

In [26]:
clean_projects[['goal', 'usd_pledged', 'backers']].corr()

,goal,usd_pledged,backers
goal,1.000000,0.006424,0.003033
usd_pledged,0.006424,1.000000,0.569047
backers,0.003033,0.569047,1.000000


From this table, you can see that the correlation coefficient between the USD pledged and the number of backers is `0.57`, which implies that there is some positive correlation between these two columns, but not between any of the other columns. This bears out as you might expect: as the number of backers increases, the amount pledged will also increase. Sometimes, though, the results can be surprising.

## Statistical testing 

The following sections will be a demonstration of testing for statistical significance in Python. If you are unfamiliar with this concept, don't worry; it will be covered later in the program, and you'll also learn how to perform these tests in Python.

### Statistical significance of correlations

It is possible to test for statistical significance of a correlation—but not in pandas. Instead, you will use the excellent [SciPy](https://docs.scipy.org/doc/scipy/reference/index.html) library for scientific computing. This library has a tremendous number of built-in statistical functions.

To conduct a Pearson correlation test, use the `stats.pearsonr()` method. Here, you are testing the likelihood of this linear relationship existing in the overall population:

In [27]:
stats.pearsonr(clean_projects['backers'], clean_projects['usd_pledged'])

(0.5690472468189988, 0.0)

This method returns two outputs: the correlation of `0.569` (which you saw before in pandas) and the p-value. In this case, the p-value is so small that SciPy rounded it to `0.0`. From this, you can expect a significant correlation between these two variables to generalize out to the population.

Statistics, however, is all about probability, so you will never know for sure that this relationship will be found in the population. That's why, when presenting a p-value in a written document or slides, it's common to format a very low p-value as `<0.05` or `<0.005` instead of `0.0`.

## Independent-samples t-test

You will be learning about the t-test in much greater depth in an upcoming module. However, it is worth doing some basic exploration in Python to see how some of it works. Suppose that you wanted to find out if baseball salaries changed between 1990 and 2000. You could start by gathering data about the salaries of players in 1990 and 2000. It is reasonable to assume that some players in 1990 would make more than some players in 2000, and vice versa. Simply using the averages is subject to too many variables, and that is where the independent-samples t-test comes in.

You will use the SciPy library for this as well. 

Now, load up a DataFrame with the data.

In [0]:
players_df = pd.read_csv("https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/players.csv")

In [29]:
players_1990 = players_df.iloc[(players_df['yearid'] == 1990).values]
players_1990.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 867 entries, 3289 to 4155
Data columns (total 14 columns):
playerid        867 non-null object
birthyear       867 non-null int64
birthcountry    867 non-null object
deathyear       36 non-null float64
namefirst       867 non-null object
namelast        867 non-null object
weight          867 non-null int64
height          867 non-null int64
bats            867 non-null object
throws          867 non-null object
yearid          867 non-null int64
teamid          867 non-null object
lgid            867 non-null object
salary          867 non-null int64
dtypes: float64(1), int64(5), object(8)
memory usage: 101.6+ KB


In [30]:
players_2000 = players_df.iloc[(players_df['yearid'] == 2000).values]
players_2000.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 836 entries, 12263 to 13098
Data columns (total 14 columns):
playerid        836 non-null object
birthyear       836 non-null int64
birthcountry    836 non-null object
deathyear       10 non-null float64
namefirst       836 non-null object
namelast        836 non-null object
weight          836 non-null int64
height          836 non-null int64
bats            836 non-null object
throws          836 non-null object
yearid          836 non-null int64
teamid          836 non-null object
lgid            836 non-null object
salary          836 non-null int64
dtypes: float64(1), int64(5), object(8)
memory usage: 98.0+ KB


You have 867 salaries from 1990 and 836 salaries from 2000. The t-test will compare the means of these two samples and give a value that indicates how different these two means are. 

The t-test will return a p-value indicating how likely you are to see such a large difference between your two sample means if the two population means are equal. A low p-value means that it is unlikely, meaning that you have a significant result. A high p-value means that it is likely that you would see such a large difference between the sample means, even if there isn't actually a difference in population means. In this case, you reject the null hypothesis that you cannot make meaningful statements about the difference.

Use the [`ttest_ind()` function](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_ind.html#scipy.stats.ttest_ind) to perform a t-test for these two samples.

The null hypothesis is that there is no difference between these two samples. A low p-value negates this null hypothesis.

In [31]:
stats.ttest_ind(players_1990['salary'], players_2000['salary'])

Ttest_indResult(statistic=-16.90651573407297, pvalue=2.1593870550905857e-59)

Note that  the negative t-value is not important; that is, you should just look at the absolute value. The sign of the t-value just indicates the directionality of the relationship between these samples. Try running the test again with the two samples swapped around; you'll see that you get the same value, but positive.

There is a difference of `16.9` standard deviations between these two means. But is this just due to random chance, or is that difference significant? The p-value is extremely small, implying that this is a real difference. Baseball salaries changed dramatically between 1990 and 2000.

# Calculating the confidence interval

The `scipy.stats` module provides some, but not all, of the results of a t-test. Although `ttest_ind()` *does* provide the test statistic and p-value, it does *not* provide measures of effect size, such as the confidence interval. To do that, you will have to crunch the numbers yourself with the help of pandas.

To calculate the confidence interval, take your mean difference plus or minus the *margin of error*, which is calculated as your standard error of the mean difference times the test statistic of `1.96`. The value of `1.96` is based on the fact that 95% of the area of a normal distribution is within 1.96 standard deviations of the mean.

Start by calculating the sample size, mean, and variance for each of your two samples. Note that by indexing the results of `.shape`, you can extract just the number of rows from the DataFrame. (The number of columns would be position `[1]`).

In [0]:
# Calculate the sample size, mean, and variance of each sample
# You will need this information to calculate standard errors

sample_1_n = players_1990.shape[0]
sample_2_n = players_2000.shape[0]
sample_1_mean = players_1990['salary'].mean()
sample_2_mean = players_2000['salary'].mean()
sample_1_var = players_1990['salary'].var()
sample_2_var = players_2000['salary'].var()

The next step requires taking a square root. To do that, you can import the `math` module. From there, you can estimate the lower and upper bounds of the confidence interval.

In [33]:
# Calculate the standard error and compute the confidence intervals

import math
std_err_difference = math.sqrt((sample_1_var/sample_1_n)+(sample_2_var/sample_2_n))

mean_difference = sample_2_mean - sample_1_mean

margin_of_error = 1.96 * std_err_difference
ci_lower = mean_difference - margin_of_error
ci_upper = mean_difference + margin_of_error

print("The difference in means at the 95% confidence interval is between "+str(ci_lower)+" and "+str(ci_upper)+".")

The difference in means at the 95% confidence interval is between 1306451.096390415 and 1655570.6584535995.


Whew! That was a lot of number-crunching! As with anything in Python, it's a good idea to write a function for a task that you expect to repeat. And considering that the [American Statistical Association](https://amstat.tandfonline.com/doi/full/10.1080/00031305.2016.1154108#.XQmJhcTPyPo) stated that "a p-value ... does not measure the size of an effect or the importance of a result," you most certainly will be estimating the confidence interval early and often! So here is the new function, `get_95_ci()`:

In [34]:
def get_95_ci(array_1, array_2):
    sample_1_n = array_1.shape[0]
    sample_2_n = array_2.shape[0]
    sample_1_mean = array_1.mean()
    sample_2_mean = array_2.mean()
    sample_1_var = array_1.var()
    sample_2_var = array_2.var()
    mean_difference = sample_2_mean - sample_1_mean
    std_err_difference = math.sqrt((sample_1_var/sample_1_n)+(sample_2_var/sample_2_n))
    margin_of_error = 1.96 * std_err_difference
    ci_lower = mean_difference - margin_of_error
    ci_upper = mean_difference + margin_of_error
    return("The difference in means at the 95% confidence interval (two-tail) is between "+str(ci_lower)+" and "+str(ci_upper)+".")

get_95_ci(players_1990['salary'],players_2000['salary'])

'The difference in means at the 95% confidence interval (two-tail) is between 1306451.096390415 and 1655570.6584535995.'

Using the confidence interval, you can quantify the difference between 2000 and 1990 as likely being between \\$1.30 million and \\$1.65 million—that's a solid effect size!